In [2]:
import subprocess
# from ruamel.yaml import YAML
# yaml = YAML(typ='safe', pure=True)
import yaml

import sys
import os
# import re
# import numpy as np
# import scipy.io as sio
# import imageio
# import tqdm
# import matplotlib.pyplot as plt
# from matplotlib.animation import FFMpegWriter


# If your script is in a subfolder, navigate one directory up
parent_dir = os.path.dirname(os.getcwd())

# Add the parent directory to sys.path
sys.path.append(parent_dir)

# from utlis.projection import *                                 
from utlis.com_traga_utlis import load_data, plot_3d_trajectory, detect_jumps, generate_jump_video#*
# from utlis.com_traga_utlis import plot_com_all, process_folders, temp_change_calib_pos_to_0, generate_dannce_vid_seq# *

In [7]:
#below are just final steps of optimization.... usually in the beginning i will just predict many 100 using the integrated debugging function in yaml file

def com_single_vis(base_folder, com_foler):
    # weired_folders = [
    # # "/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_07_03/1691486_left_right_caffeine_1448",
    # # "/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_07_03/1691486_left_caffeine_1051",
    # # "/home/lq53/mir_data/24summ/2024_07_10/1691485_no_hole_habituation_13_59"
    # "/home/lq53/mir_data/24summ/2024_06_26/1686940_left"
    # ]
    # for wie in weired_folders:
        # /home/lq53/mir_data/24summ/2024_06_26/1686940_left/COM_df/predict_results//vis/vis_combined_Camera2_1000_from_0.mp4
    # com_foler = os.path.join(wie, 'COM_df/predict_results/df_labels_240816_6comsets_continued')
    com_path = os.path.join(com_foler, 'com3d.mat')   
    com_folder_save = os.path.join(com_foler, 'vis')
    if not os.path.exists(com_folder_save):
        os.makedirs(com_folder_save)
    graph_title = "z_com3d_vis_"
    
    com_data = load_data(com_path)

    plot_3d_trajectory(com_data, graph_title, com_folder_save)
    jump_indices = detect_jumps(com_data, com_folder_save)

    # # # produce video, which is not necessary if not labeling more com to detect what's wrong


    save_path = com_folder_save # os.path.join(com_foler, 'vis') #os.path.join(pred_path, 'vis')
    # if not os.path.exists(save_path):
    #     os.makedirs(save_path)
    # base_folder = wie
    # base_base_folder = os.path.dirname(base_folder)
    generate_jump_video(com_data, base_folder, jump_indices, graph_title, save_path, cam='Camera1')


def update_yaml(path, param_changes):
    if not os.path.exists(path):
        print("file not existed", path)
    with open(path, 'r') as file:
        config = yaml.safe_load(file)
    if config is None:
        print(config)
    # Apply the parameter changes
    config.update(param_changes)
    # for key, value in param_changes.items():
    #     if key in config:
    #         config[key] = value

    # Save the updated configuration back to the same file
    with open(path, 'w') as file:
        yaml.safe_dump(config, file)

# def run_command(command, env_name):
#         """Run a command in a specific environment."""
#         activate_env = f"source activate {env_name} && " if env_name else ""
#         deactivate_env = " && conda deactivate" if env_name else ""
#         full_command = f"{activate_env}{command}{deactivate_env}"
#         subprocess.run(full_command, shell=True, check=True, text=True)

def COM_train_pred_vis(base_fold, io_param_changes, param_changes=None):
    com = 'com'
    # dannce = 'dannce' #actually dannce probably will be different so lets do this later

    io_name = 'mir_io.yaml'
    io_path = os.path.join(base_fold, io_name)
    # print(io_path)

    com_config_name = f'mir_{com}_mouse_config.yaml'
    com_config_path = os.path.join(base_fold, com_config_name)

    if io_param_changes is not {}:
        # print(io_path)
        update_yaml(io_path, io_param_changes)
        print('updated io file')

    if param_changes is not {}:
        update_yaml(com_config_path, param_changes)
        print('updated com_config file')
    
    # activate_dannce_env = f"source activate dannce && "
    # activate_bbop_env = f"source activate BBOP && "
    # deactivate_env = " && conda deactivate"

    print("Starting training...")
    try:
        subprocess.run(["conda", "run", "-n", "dannce", "com-train", com_config_path], check=True, text=True)
        # Activate the environment
        # subprocess.run(["source", "activate", "dannce"], shell=True, check=True, text=True)
        
        # Run the training command
        # subprocess.run(["com-train", com_config_path], check=True, text=True)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e}")
    # {activate_dannce_env}
    # subprocess.run(f"{activate_dannce_env} com-train {com_config_path}", shell=True, check=True, text=True)
    print("Training completed.")

    print("Starting prediction...")
    try:
        subprocess.run(["conda", "run", "-n", "dannce", "com-predict", com_config_path], check=True, text=True)
        # Activate the environment
        # subprocess.run(["source", "activate", "dannce"], shell=True, check=True, text=True)
        # Run the training command
        # subprocess.run(["com-predict", com_config_path], check=True, text=True)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e}") 
    # {deactivate_env}
    # subprocess.run(f"com-predict {com_config_path} {deactivate_env}", shell=True, check=True, text=True)
    print("Prediction completed.")

    print('start visulizing')
    com_single_vis(base_fold, io_param_changes['com_predict_dir'])
    print(f'finished, check results in{io_param_changes["com_predict_dir"]}')



    


In [8]:
base_fold = "/home/lq53/mir_data/24summ/2024_06_26/1686940_left"

# new_train_dir_name = './COM_df_z/train_results/df_z_240816_6comsets_fromdemo/'
param_changes = {
    # 'max_num_samples': 1000 # 'max' 
    "epochs": 1
    # com_finetune_weights:
}

io_param_changes = {
    'com_train_dir': './COM_df_z/train_results/df_z_240823_6comsets_fromdemo_1epotest/', # new_train_dir_name,  # Example: change the learning rate
    'com_predict_dir': './COM_df_z/predict_results/df_z_240823_6comsets_fromdemo_1epotest/', 
    # 'com_train_dir': './COM_df_z/train_results/df_z_240823_6comsets_fromdemo/', # new_train_dir_name,  # Example: change the learning rate
    # 'com_predict_dir': './COM_df_z/predict_results/df_z_240823_6comsets_fromdemo/',        # Example: change the batch size
    # 'com_predict_weights': 
    # 'com_file': 
    # dannce_predict_model
    # dannce_train_dir: ./DANNCE/train_results/df_240809_mirlabelall_2nd100epo_opti/
# dannce_predict_dir: ./DANNCE/predict_results/240812_ssh_2bestwalk/
}




COM_train_pred_vis(base_fold, io_param_changes, param_changes)

updated io file
updated com_config file
Starting training...


Traceback (most recent call last):
  File "/home/lq53/miniconda3/envs/dannce/bin/com-train", line 33, in <module>
    sys.exit(load_entry_point('dannce', 'console_scripts', 'com-train')())
  File "/home/lq53/mir_repos/dannce/dannce/cli.py", line 122, in com_train_cli
    params = build_clarg_params(args, dannce_net=False, prediction=False)
  File "/home/lq53/mir_repos/dannce/dannce/cli.py", line 164, in build_clarg_params
    params = build_params(args.base_config, dannce_net)
  File "/home/lq53/mir_repos/dannce/dannce/interface.py", line 82, in build_params
    params = processing.read_config(base_params["io_config"])
  File "/home/lq53/mir_repos/dannce/dannce/engine/processing.py", line 1207, in read_config
    with open(filename) as f:
FileNotFoundError: [Errno 2] No such file or directory: 'mir_io.yaml'

ERROR conda.cli.main_run:execute(125): `conda run com-train /home/lq53/mir_data/24summ/2024_06_26/1686940_left/mir_com_mouse_config.yaml` failed. (See above for error)


Error occurred: Command '['conda', 'run', '-n', 'dannce', 'com-train', '/home/lq53/mir_data/24summ/2024_06_26/1686940_left/mir_com_mouse_config.yaml']' returned non-zero exit status 1.
Training completed.
Starting prediction...
Error occurred: Command '['conda', 'run', '-n', 'dannce', 'com-predict', '/home/lq53/mir_data/24summ/2024_06_26/1686940_left/mir_com_mouse_config.yaml']' returned non-zero exit status 1.
Prediction completed.
start visulizing


Traceback (most recent call last):
  File "/home/lq53/miniconda3/envs/dannce/bin/com-predict", line 33, in <module>
    sys.exit(load_entry_point('dannce', 'console_scripts', 'com-predict')())
  File "/home/lq53/mir_repos/dannce/dannce/cli.py", line 110, in com_predict_cli
    params = build_clarg_params(args, dannce_net=False, prediction=True)
  File "/home/lq53/mir_repos/dannce/dannce/cli.py", line 164, in build_clarg_params
    params = build_params(args.base_config, dannce_net)
  File "/home/lq53/mir_repos/dannce/dannce/interface.py", line 82, in build_params
    params = processing.read_config(base_params["io_config"])
  File "/home/lq53/mir_repos/dannce/dannce/engine/processing.py", line 1207, in read_config
    with open(filename) as f:
FileNotFoundError: [Errno 2] No such file or directory: 'mir_io.yaml'

ERROR conda.cli.main_run:execute(125): `conda run com-predict /home/lq53/mir_data/24summ/2024_06_26/1686940_left/mir_com_mouse_config.yaml` failed. (See above for error)


FileNotFoundError: [Errno 2] No such file or directory: './COM_df_z/predict_results/df_z_240823_6comsets_fromdemo_1epotest/com3d.mat'